In [4]:
import pathlib
import pandas as pd
import numpy as np
import yaml
import os

import plotly.express as px
import plotly.graph_objects as go



###	데이터 정합성 체크
#####	1. Data type, non-null count check (주의: int 여도 사실상 categorical data 일 수 있음)

In [6]:
df = pd.read_csv(r'C:\Users\whyyo\Downloads\git_link\02_pydsp_tutorial\result_df_head10.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 48 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   plate_name       10 non-null     object 
 1   device           10 non-null     object 
 2   consumable_name  10 non-null     object 
 3   is_sample        10 non-null     object 
 4   well             10 non-null     object 
 5   channel          10 non-null     object 
 6   temperature      10 non-null     object 
 7   sequence         10 non-null     object 
 8   variable1        10 non-null     int64  
 9   variable2        10 non-null     int64  
 10  variable3        10 non-null     int64  
 11  variable4        10 non-null     int64  
 12  variable5        10 non-null     int64  
 13  variable6        10 non-null     int64  
 14  variable7        10 non-null     float64
 15  variable8        10 non-null     int64  
 16  variable9        10 non-null     int64  
 17  variable10       10

In [8]:
# treanspose df

result_df_1 = df.head(1).T

In [9]:
df_info = {
    'column': df.columns,
    'Non-Null': df.count(),
    'Dtype': df.dtypes,
    # 'value_sample': df.head(1).T
}

In [10]:
# column 별 있는 값의 개수 (Non-Null), Data type, 값의 예시를 들어주는 정보를 담은 table
pd.concat([pd.DataFrame(df_info), df.head(1).T], axis= 1)

,column,Non-Null,Dtype,0
plate_name,plate_name,10,object,plate1
device,device,10,object,device1
consumable_name,consumable_name,10,object,consumable1
is_sample,is_sample,10,object,Sample
well,well,10,object,A01
channel,channel,10,object,channel_1
temperature,temperature,10,object,Low
sequence,sequence,10,object,[6004.03532629 5969.00735142 5943.7098278 593...
variable1,variable1,10,int64,0
variable2,variable2,10,int64,0


In [11]:
# split discrete, continuous variables

cat_col = [col for col in df.columns if df[col].dtypes in ['object', 'int64']]
num_col = [col for col in df.columns if df[col].dtypes in ['float64']]

In [12]:
# cat_col, num_col 이 잘 정의 되었는지 하나 하나 확인 해야 함. 
# 잘 됐겠거니 하고 넘어가지 말 것. 
cat_col

['plate_name',
 'device',
 'consumable_name',
 'is_sample',
 'well',
 'channel',
 'temperature',
 'sequence',
 'variable1',
 'variable2',
 'variable3',
 'variable4',
 'variable5',
 'variable6',
 'variable8',
 'variable9',
 'variable11',
 'variable12',
 'variable13',
 'variable15',
 'variable16',
 'variable18',
 'variable19',
 'variable20',
 'variable25',
 'variable26',
 'variable28',
 'variable29',
 'variable30',
 'variable31',
 'variable32',
 'variable33']

In [13]:
num_col 

['variable7',
 'variable10',
 'variable14',
 'variable17',
 'variable21',
 'variable22',
 'variable23',
 'variable24',
 'variable27',
 'variable34',
 'variable35',
 'variable36',
 'variable37',
 'variable38',
 'variable39',
 'variable40']

##### 2. Duplicity check

In [16]:
# duplicity check => sum == 0 => no duplicity 
df.duplicated(subset=['well','channel','temperature']).sum()

0

##### 3. Column 별 discrete 한 값을 가질 경우, unique 값 조회

In [17]:
# column 별 unique value 확인

for col in df[cat_col].columns:
    print(col)
    print(df[col].unique())
    # print line change
    print('\n')

for col in df[cat_col].columns:
    print(col)
    print(df[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) else x).unique())
    # print line change
    print('\n')

plate_name
['plate1' 'plate2' 'plate3' 'plate4' 'plate5' 'plate6' 'plate7' 'plate8'
 'plate9' 'plate10']


device
['device1' 'device2' 'device3' 'device4' 'device5' 'device6' 'device7'
 'device8' 'device9' 'device10']


consumable_name
['consumable1' 'consumable2' 'consumable3' 'consumable4' 'consumable5'
 'consumable6' 'consumable7' 'consumable8' 'consumable9' 'consumable10']


is_sample
['Sample']


well
['A01' 'A02']


channel
['channel_1' 'channel_2' 'channel_3' 'channel_4' 'channel_5' 'channel_6'
 'channel_7' 'channel_8' 'channel_9' 'channel_10']


temperature
['Low' 'High']


sequence
['[6004.03532629 5969.00735142 5943.7098278  5931.63602259 5919.77746832\n 5902.41944855 5889.90725369 5887.45881499 5883.70834851 5871.42398282\n 5870.94366861 5863.9111029  5867.03602087 5863.18233238 5861.17090112\n 5860.36908262 5850.68013236 5858.08715422 5854.59832541 5859.96304537\n 5872.61283127 5900.62887147 5930.39522773 5991.23269329 6082.91960062\n 6189.95407865 6336.88232452 6508.503734

##### 4. outlier 존재 여부 체크 (univariate)

In [18]:
# numerical columns only
# IQR, LOF, isolation forest, autoencoder 등 다양한 방법이 있지만 본 예제에서는 IQR 로 체크

import plotly.express as px

fig = px.box(df[num_col])
fig.show()


##### 5. 논리적으로 (domain knowledge 상) 유효하지 않은 값 체크

In [36]:
# 예를 들어, varaible27 값의 정합성 체크 후 기준에 맞지 않는 row 삭제는 아래와 같이 진행
# 가정: variable27 의 범위는 -1 이상 45 이하, null 값이 오면 안됨 

# Check Range
is_out_of_range = (df['variable27'] < -1) | (df['variable27'] > 45)

# Check for NaN values
is_nan = df['variable27'].isna()

# Check for infinite values
is_inf = np.isinf(df['variable27'])

# Check for valid values (not NaN or infinite)
is_valid = ~is_nan & ~is_inf & ~is_out_of_range

# Filter the DataFrame to get valid rows
valid_data = df[is_valid]

##### 6. Distribution of variables
#### 6.1. Univariate (변수가 1개인경우) 분포 시각화

In [40]:
#### Univariate (변수가 1개인 경우) 분포 시각화
#### variable27 변수 기준으로 이의 값 별 frequency 확인
fig = px.histogram(data_frame = df
            ,x = 'variable27'
            ,nbins = 10
            )
fig.show()

#### bivariate (변수가 2개인 경우) 분포 시각화
bivariate distribution
- categorical & continuous: box plot, violin plot
- continuous & continuous: scatter plot

In [26]:
px.box(df, x='well', y='variable27', points='all')

In [34]:
px.scatter(df, x= 'variable7', y='variable10', color='well')